In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time

In [2]:
query="""
With microsegment as (

SELECT DISTINCT c.customer_id AS user_id,
                c.site_type AS region_code_ms,
                
                (case 
                    -- Colombia
                    when m.microsegment_id not in (1,3,4,28,5,6,7,8,2) and c.site_type = 'BOG' then 'Sin Microsegmento'
                    when m.microsegment_id not in (1,3,4,28,5,11,10,9,2,12) and c.site_type = 'BAQ' then 'Sin Microsegmento'
                    when m.microsegment_id not in (3,4,28,1,24,11,5,2,27) and c.site_type = 'MDE' then 'Sin Microsegmento'
    
                    -- Mexico
                    when m.microsegment_id not in (1,13,14,30,15,16,29,17,6,7,2) and c.site_type = 'CMX' then 'Sin Microsegmento'
                    when m.microsegment_id not in (1, 13,14,30, 15,16,29, 6, 33, 2, 31) and c.site_type = 'GDL' then 'Sin Microsegmento'
                    when m.microsegment_id not in (13,14,30, 15,16,29, 32,2,6,33) and c.site_type = 'PBC' then 'Sin Microsegmento'
                    
                    -- Brasil
                    when m.microsegment_id not in (21,34,38,37,35,36,39,26) and c.site_type in ('SPO','BHZ','CWB','VCP') then 'Sin Microsegmento'
                        
                else m.description end) as microsegment_ok,
                
                m.microsegment_id,
                mo.name as ms_method
                
                
FROM postgres_main_federate."growth_microsegmentation.customer_microsegment" AS cm
INNER JOIN postgres_main_federate."growth_microsegmentation.microsegment" m ON cm.microsegment_id = m.id
INNER JOIN postgres_main_federate."growth_microsegmentation.customer" c ON cm.customer_id = c.id
INNER JOIN postgres_main_federate."growth_microsegmentation.models" mo ON mo.id = cm.model_type_id


)

select m.user_id,
m.region_code_ms,
--m.microsegment_id,

(case 
        when m.microsegment_ok in ('Corrientazo c/desayuno','Corrientazo s/desayuno','Tienda/Restaurante') then 'Corrientazo' 
        when m.microsegment_ok in ('Fonda c/desayuno','Fonda s/desayuno') then 'Fonda'
        when m.microsegment_ok in ('Taquería en calle','Taquería en local') then 'Taquería'

else m.microsegment_ok end) as microsegment,

m.ms_method

from microsegment m
where m.microsegment_ok != 'Sin Microsegmento'
"""

segmentos_fulll = read_connection_data_warehouse.runQuery(query)

segmentos_fulll.columns = ['customer_id', 'city', 'microsegment','NA']
segmentos_fulll = segmentos_fulll[['customer_id', 'city', 'microsegment']]

In [3]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros
"""Todays date should be the first day of the next month which is going to begin with SD"""
city_code = 'BAQ'
    #year-month-day
todays_date = '2023-03-01' 
non_buyers_date = '2023-02-01' #1 months
customer_date_desde = '2023-02-01' #3 months
    #data for sku sales data
sku_date_desde = '2023-01-01' #2 months

t_cambio = 1 #COL=3000 // MX=19.19 // BR=3.88

### Loading COSTUMERS INFO

In [4]:
#//--------------------------------------
#//Ruta de data // Base Data (Without Deleting)
if city_code in ('BAQ', 'BOG', 'CMX', 'SPO','MDE','GDL','PBC','CWB','BHZ','VCP'):
    segmentos_full = segmentos_fulll[segmentos_fulll.city == city_code].copy()
else:
    data = [[city_code,'NA',0]]
    segmentos_full = pd.DataFrame(data, columns=['city', 'microsegment', 'customer_id'])

In [5]:
#//--------------------------------------
#//Ruta de data // KAM's costumers to delete
#//KAMs customers me lo manda Emiliano Lomeli (SE ACTUALIZAN DE UN SCRIPT MES A MES)
users_eliminar_kam = pd.read_csv("SD/import_data/segments/KAMs_NOV.csv")['costumer_id'].unique().tolist()

In [6]:
#//Users to ignore  ->>> ACAA [(ACA IMPORTANTE VER EL SEGMENTOS_PREV2 QUE ESTAS TRAYENDO)]
#//At the start of the month we just delete KAM users, then Users that already got SD
users_eliminar = users_eliminar_kam #+ segmentos_prev1['customer_id'].unique().tolist() #+ segmentos_prev2['customer_id'].unique().tolist() + segmentos_prev3['customer_id'].unique().tolist()

### Loading FORCED_SKUS // Cleaning SEGMENT_NAMES

In [7]:
data = [['NA','NA','NA',0]]
forced_skus = pd.DataFrame(data, columns=['microsegment', 'subcat', 'name', 'sku_ids'])

### Loading DATA_VENTAS // Removing SKUs /\ CUSTOMERS_ID

In [8]:
#//Remove skus 
#It is used because some SKUs can get in trouble with supplier for the SD or to exclude them for another activity 
skus_remove = [
-303030,
63697,
280865,
280868,
280862,
440367,
-303057,
332389,
271602,
83969,
126139,
191539,
123039,
120813,
241989,
323989,
323995,
323998,
271594,
262884,
333300,
-301818,
285353,
285344,
296188,
118027,
118025,
111139,
31507,
118020,
82996,
102355,
90176,
266755,
489253,
99017,
490568,
489259,
254112,
254113,
147341,
489257,
147339,
56612,
324145,
401540,
56606,
324143,
615661,
613098,
489263,
322945,
333294,
333296,
489261,
333298,
333292,
126505,
290222,
162843,
37188,
37211,
377868,
285338,
296040,
296041,
367096,
367101,
367142
]

In [9]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",48322,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [10]:
good_skus = frida_products.sku_id.unique()

In [11]:
def detectSKUs(skus, strs):
    
    if  str(strs)=='nan':
        return True #To exclude nans    
    for sku in skus:
        if str(sku) in strs:
            return True
    return False

In [12]:
#Loading SELL'S DATA from a function
data_ventas = data_ventas_query(ciudad=city_code, tipo_cambio=t_cambio, desde=customer_date_desde, hasta = todays_date, tipo_negocio=1)

In [13]:
data_ventas = data_ventas[~data_ventas['sku_id'].isin(skus_remove)]

In [14]:
data_ventas = data_ventas[data_ventas['sku_id'].isin(good_skus)]

In [15]:
#//customers for offers */(has further uses)*/
microsegmentacion_df = segmentos_full[(~segmentos_full['customer_id'].isin(users_eliminar))]
data_ventas = data_ventas[(~data_ventas['customer_id'].isin(users_eliminar))]

### Create the MS "Other" for BAU 

In [16]:
data_full = data_ventas.merge(microsegmentacion_df[['customer_id','microsegment']], how = 'left', on = 'customer_id')

In [17]:
data_full['microsegment'] = data_full['microsegment'].fillna('Other')

In [18]:
data_full[['customer_id','microsegment']].groupby(['microsegment']).nunique().reset_index()

,microsegment,customer_id
0,Boteco,103
1,Grill/Churrascaria/Espetinho,107
2,Japonesa/Internacional,41
3,Lanches,524
4,Other,167
5,Padaria/Cafe/Bolos,485
6,Pizzaria/Pastelaria/Italiano,127
7,Pratos/Refeições,590
8,Sucos/Açaí,118


In [19]:
segments_validation = data_full[['customer_id','microsegment']].groupby(['microsegment']).nunique().reset_index()

In [20]:
for ms in segments_validation[segments_validation.customer_id < 180]['microsegment'].unique():
    data_full['microsegment'] = data_full['microsegment'].str.replace(ms,'Other')

In [21]:
data_full[['customer_id','microsegment']].groupby(['microsegment']).nunique().reset_index()

,microsegment,customer_id
0,Lanches,524
1,Other,663
2,Padaria/Cafe/Bolos,485
3,Pratos/Refeições,590


In [22]:
#Get the same variable to do the process job
data_ventas = data_full

### PARAMETERS // SOME FUNCTIONS - GET READY TO PROCESSING

In [23]:
def ms_data(city_code):

    from procurement_lib import redash, dw, send_message, send_slack_notification, GoogleSheet, logging
    
    paramss_csv=GoogleSheet("1k0ckgkLddxc5lY8NUVGCkvDLTuulLaJm-N535oMNZNw")
    city_paramss = paramss_csv.get_as_dataframe(city_code)
    
    parametros = {}
    parametros[city_code] = {}
    
    for i,ms in enumerate(city_paramss.iloc[:,4:].columns):
        parametros[city_code][ms] = [(value[1],value[2],value[3],value[4+i]) for value in city_paramss.values if value[4+i]>0]
        
    return parametros

In [24]:
parametros = ms_data(city_code)

In [25]:
def normalize_text(raw_text):
    """
    Removes common accent characters and normalize text.
    """
    raw_text = raw_text.replace(" ", "")
    raw_text = raw_text.replace("�", "")
    raw_text = raw_text.replace("/", "")
    raw_text = re.sub(r'\s*', '', raw_text)
    raw_text = re.sub(r"[àáâãäå]", 'a', raw_text)
    raw_text = re.sub(r"[èéêë]", 'e', raw_text)
    raw_text = re.sub(r"[ìíîï]", 'i', raw_text) 
    raw_text = re.sub(r"[òóôõö]", 'o', raw_text)
    raw_text = re.sub(r"[ùúûü]", 'u', raw_text)
    raw_text = re.sub(r"[ýÿ]", 'y', raw_text)
    raw_text = re.sub(r"[ß]", 'ss', raw_text)
    raw_text = re.sub(r"[ñ]", 'n', raw_text)
    raw_text = re.sub(r"[ç]", 'c', raw_text)
    raw_text = raw_text.lower()
    return raw_text

In [26]:
#Sacar de aca los microsegmentos
msg = data_full.microsegment.unique()
msg

array(['Padaria/Cafe/Bolos', 'Lanches', 'Other', 'Pratos/Refeições'],
      dtype=object)

### END

In [27]:
#//Variables to store all the segments and offers of the city
global_offer_csv = pd.DataFrame()
global_segment_list = [] #//for procesing

In [28]:

#//----------------------------------------------------------------------------
#//Big loop for creating each city segments/offers files
for microsegment in msg:
    print('//-------------------------------------------------------------------------------')
    print(microsegment)

    #//Forced sku per microsegment
    forced_skus_formated = forced_skus[forced_skus['microsegment']==microsegment].groupby('subcat')['sku_ids'].unique().to_dict()
    #//Customers per microsegment
    customers_ms = data_ventas[(data_ventas['microsegment'] == microsegment)]['customer_id'].unique()
    #//Customers to give them the skus
    customers_for_skus_ms =  data_ventas[['microsegment','customer_id']][(data_ventas.microsegment == microsegment) & (~data_ventas.customer_id.isin(users_eliminar))]['customer_id'].unique()
    
    #----------------------------------------------------------- START OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    
    
    #//----------------------------------------------------------------------------------------------------------------------------
    ms_name = normalize_text(microsegment) #NORMALIZE TEXT FORMAT
    #//----------------------------------------------------------------------------------------------------------------------------
    
    data_buyers = data_ventas[data_ventas['customer_id'].isin(customers_ms)].copy()
    print(data_buyers.customer_id.nunique())
    data_skus = data_ventas[data_ventas['customer_id'].isin(customers_for_skus_ms)].copy()


    subcats = list(map(lambda x: x[0], parametros[city_code][microsegment]))
    max_uses = list(map(lambda x: x[1], parametros[city_code][microsegment]))
    max_order = list(map(lambda x: x[2], parametros[city_code][microsegment]))
    discount = list(map(lambda x: x[3], parametros[city_code][microsegment]))

    #----------------------------------------------------------- END OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    #----------------------------------------------------------- START OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    a, b, c = proceso_para_csv(data_ventas=data_buyers, todays_date=todays_date, 
                non_buyers_date=non_buyers_date, city=city_code, list_to_filter=subcats, offer_discount=discount, offer_max_uses=max_uses, offer_max_order=max_order, level='subcat', additional_words=ms_name,
                          limite=0.75, data_ventas_skus = data_skus, forzar_skus = forced_skus_formated)
    
    print(b.nunique())
    #//appending to global data frame
    global_offer_csv = global_offer_csv.append(a)
    
    #//Data formating
    lst = []
    name = 'export_data/MS/nb_'+city_code+'_'+ms_name+'.txt'
    for seg in b.columns:
        id_list = b[seg].dropna().astype(int)
        d = {'name':seg, 'customersIds': list(id_list)}
        lst.append(d)
        
           
    #//appending to global list    
        global_segment_list.append(d)
    
    print(f'Done {city_code} {ms_name}')
    #----------------------------------------------------------- END OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    
#//End of big loop and start of data bundle    

//-------------------------------------------------------------------------------
Padaria/Cafe/Bolos
485
Uso de data externa
20230301_CWB_padariacafebolos_NB_aguaseisotonicos_W9              399
20230301_CWB_padariacafebolos_NB_cervejas_W9                      352
20230301_CWB_padariacafebolos_NB_refrigeranteseenergeticos_W9     389
20230301_CWB_padariacafebolos_NB_sucosechas_W9                    460
20230301_CWB_padariacafebolos_NB_destilados_W9                    473
20230301_CWB_padariacafebolos_NB_alimentoscongelados_W9           467
20230301_CWB_padariacafebolos_NB_batataspre-fritas_W9             441
20230301_CWB_padariacafebolos_NB_polpaseaai_W9                    462
20230301_CWB_padariacafebolos_NB_coposrecipientesepratos_W9       473
20230301_CWB_padariacafebolos_NB_embalagens_W9                    479
20230301_CWB_padariacafebolos_NB_papeis_W9                        418
20230301_CWB_padariacafebolos_NB_sacos_W9                         481
20230301_CWB_padariacafebolos_NB_ut

In [29]:
# #//--------------------------------------------
# #//Exporting single file data
# global_offer_csv.to_csv(f'SD/export_data/MS/csv_{city_code}_All_MS.csv', index = False)

# with open(f'SD/export_data/MS/nb_{city_code}_All_MS.txt', 'w') as outfile:
#         json.dump(global_segment_list, outfile)    
# with open(f'SD/export_data/MS/nb_{city_code}_All_MS.json', 'w') as outfile:
#         json.dump(global_segment_list, outfile)
# print(f'Done {city_code}')

# Aca Empieza la Experimentacion

In [30]:
from sklearn.model_selection import train_test_split 

In [31]:
#//Variables to store all the segments and offers of the city
global_offer_csv = pd.DataFrame()
global_segment_list = [] #//for procesing

In [32]:
#//----------------------------------------------------------------------------
#//Big loop for creating each city segments/offers files
aux_df = pd.DataFrame()
aux_df['aux']=5000*[0]
for microsegment in msg:
    print('//-------------------------------------------------------------------------------')
    print(microsegment)

    #//Forced sku per microsegment
    forced_skus_formated = forced_skus[forced_skus['microsegment']==microsegment].groupby('subcat')['sku_ids'].unique().to_dict()
    #//Customers per microsegment
    customers_ms = data_ventas[(data_ventas['microsegment'] == microsegment)]['customer_id'].unique()
    #//Customers to give them the skus
    customers_for_skus_ms =  data_ventas[['microsegment','customer_id']][(data_ventas.microsegment == microsegment) & (~data_ventas.customer_id.isin(users_eliminar))]['customer_id'].unique()
    
    #----------------------------------------------------------- START OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    
    
    #//----------------------------------------------------------------------------------------------------------------------------
    ms_name = normalize_text(microsegment)+"_PILOTO" #NORMALIZE TEXT FORMAT
    #//----------------------------------------------------------------------------------------------------------------------------
    
    #// Separation of types of sales (non_buyers/skus for offers)
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    rand=1
    customer_ms_2, customer_ms_1 = train_test_split(customers_ms, test_size = 0.80, random_state=rand)
    
    while abs(1-(data_ventas[data_ventas['customer_id'].isin(customer_ms_2)]['gmv_usd'].mean()/data_ventas[data_ventas['customer_id'].isin(customer_ms_1)]['gmv_usd'].mean())) > 0.05:
        #print(data_ventas[data_ventas['customer_id'].isin(customer_ms_1)]['gmv_usd'].mean(),data_ventas[data_ventas['customer_id'].isin(customer_ms_2)]['gmv_usd'].mean(),abs(1-(data_ventas[data_ventas['customer_id'].isin(customer_ms_2)]['gmv_usd'].mean()/data_ventas[data_ventas['customer_id'].isin(customer_ms_1)]['gmv_usd'].mean())))
        rand+=1
        customer_ms_2, customer_ms_1 = train_test_split(customers_ms, test_size = 0.80, random_state = rand)
        
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    
    data_buyers = data_ventas[data_ventas['customer_id'].isin(customer_ms_1)].copy() #le quite la s!!
    
    print(data_buyers.customer_id.nunique())
    data_skus = data_ventas[data_ventas['customer_id'].isin(customers_for_skus_ms)].copy()
    
    subcats = list(map(lambda x: x[0], parametros[city_code][microsegment]))
    max_uses = list(map(lambda x: int(x[1]), parametros[city_code][microsegment]))
    max_order = list(map(lambda x: int(x[2]), parametros[city_code][microsegment]))
    discount = list(map(lambda x: int(x[3]), parametros[city_code][microsegment]))

    #----------------------------------------------------------- END OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    #----------------------------------------------------------- START OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    a, b, c = proceso_para_csv(data_ventas=data_buyers, todays_date=todays_date, 
            non_buyers_date=non_buyers_date, city=city_code, list_to_filter=subcats, offer_discount=discount, offer_max_uses=max_uses, offer_max_order=max_order, level='subcat', additional_words=ms_name,
                        limite=0.75, data_ventas_skus = data_skus, forzar_skus = forced_skus_formated)

    print(b.nunique())
    #//appending to global data frame
    global_offer_csv = global_offer_csv.append(a)
    
    #//Data formating
    lst = []
    name = 'export_data/MS/nb_'+city_code+'_'+ms_name+'.txt'
    for seg in b.columns:
        id_list = b[seg].dropna().astype(int)
        d = {'name':seg, 'customersIds': list(id_list)}
        aux_df.loc[range(len(list(id_list))),seg] = list(id_list)

        lst.append(d)
        
    #//appending to global list    
        global_segment_list.append(d)
    
    print(f'Done {city_code} {ms_name}')

#----------------------------------------------------------- END OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------
#////////////-------------------------------------------Here We Start With The Second Group-----------------------------------------------------------------------------------------------------------------


#----------------------------------------------------------- START OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    
    
    #//----------------------------------------------------------------------------------------------------------------------------
    ms_name = normalize_text(microsegment)+"_CONTROL" #NORMALIZE TEXT FORMAT
    #//----------------------------------------------------------------------------------------------------------------------------
    
    #// Separation of types of sales (non_buyers/skus for offers)
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    data_buyers = data_ventas[data_ventas['customer_id'].isin(customer_ms_2)].copy() #le quite la s!!
    #//////////////////////////////// THIS IS FOR CREATE EXPERIMENTS //////////////////////////////////////////////
    
    print('//-------------------------------------------------------------------------------')
    print(f'{microsegment} Experimento50%')
    print(data_buyers.customer_id.nunique())
    data_skus = data_ventas[data_ventas['customer_id'].isin(customers_for_skus_ms)].copy()

    #----------------------------------------------------------- END OF GETTING READY THE DATA --------------------------------------------------------------------------------------
    #----------------------------------------------------------- START OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    e, f, g = proceso_para_csv(data_ventas=data_buyers, todays_date=todays_date, 
                non_buyers_date=non_buyers_date, city=city_code, list_to_filter=subcats, offer_discount=discount, offer_max_uses=max_uses, offer_max_order=max_order, level='subcat', additional_words=ms_name,
                          limite=0.75, data_ventas_skus = data_skus, forzar_skus = forced_skus_formated)
      
    print(f.nunique())
    #//appending to global data frame
    global_offer_csv = global_offer_csv.append(e)
    
    #//Data formating
    lst = []
    name = 'export_data/MS/nb_'+city_code+'_'+ms_name+'.txt'
    for seg in f.columns:
        id_list = f[seg].dropna().astype(int)
        h = {'name':seg, 'customersIds': list(id_list)}
        aux_df.loc[range(len(list(id_list))),seg] = list(id_list)
        
        lst.append(h)
        
        
    #//appending to global list    
        global_segment_list.append(h)
    
    print(f'Done {city_code} {ms_name}')
    #----------------------------------------------------------- END OF THE GENERATION OF OFFERS --------------------------------------------------------------------------------------
    
#//End of big loop and start of data bundle 

//-------------------------------------------------------------------------------
Padaria/Cafe/Bolos
388
Uso de data externa
20230301_CWB_padariacafebolos_piloto_NB_aguaseisotonicos_W9              315
20230301_CWB_padariacafebolos_piloto_NB_cervejas_W9                      285
20230301_CWB_padariacafebolos_piloto_NB_refrigeranteseenergeticos_W9     303
20230301_CWB_padariacafebolos_piloto_NB_sucosechas_W9                    366
20230301_CWB_padariacafebolos_piloto_NB_destilados_W9                    378
20230301_CWB_padariacafebolos_piloto_NB_alimentoscongelados_W9           372
20230301_CWB_padariacafebolos_piloto_NB_batataspre-fritas_W9             353
20230301_CWB_padariacafebolos_piloto_NB_polpaseaai_W9                    370
20230301_CWB_padariacafebolos_piloto_NB_coposrecipientesepratos_W9       378
20230301_CWB_padariacafebolos_piloto_NB_embalagens_W9                    382
20230301_CWB_padariacafebolos_piloto_NB_papeis_W9                        332
20230301_CWB_padariacafebolo

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


Done CWB lanches_PILOTO
//-------------------------------------------------------------------------------
Lanches Experimento50%
104
Uso de data externa
20230301_CWB_lanches_control_NB_aguaseisotonicos_W9               87
20230301_CWB_lanches_control_NB_cervejas_W9                       64
20230301_CWB_lanches_control_NB_refrigeranteseenergeticos_W9      75
20230301_CWB_lanches_control_NB_sucosechas_W9                     93
20230301_CWB_lanches_control_NB_destilados_W9                     97
20230301_CWB_lanches_control_NB_alimentoscongelados_W9            97
20230301_CWB_lanches_control_NB_batataspre-fritas_W9              79
20230301_CWB_lanches_control_NB_polpaseaai_W9                     96
20230301_CWB_lanches_control_NB_coposrecipientesepratos_W9       100
20230301_CWB_lanches_control_NB_embalagens_W9                    102
20230301_CWB_lanches_control_NB_papeis_W9                         87
20230301_CWB_lanches_control_NB_sacos_W9                         104
20230301_CWB_lanche

## Aca creo el archivo para R&D

In [33]:
# customers =  pd.DataFrame()
# for column,i in enumerate(aux_df.columns):
#     #if ('verduras' in i or 'frutas' in i or 'arroz' in i or 'aceitesygrasas' in i or 'harinasymezclas' in i or 'azucaryendulzantes' in i or 'pollo' in i or 'huevos' in i) and not 'frutasjugo' in i:
#     #if ('verduras' in i or 'frutas' in i) and not 'frutasjugo' in i:
#     customers[i] = aux_df[i]

In [34]:
# customers.iloc[:5000].to_csv(f'SD/export_data/MS/csv_{city_code}_CLIENTS_PRUEBA_R&D.csv', index = False)

# with open(f'SD/export_data/MS/nb_{city_code}_All_MS.txt', 'w') as outfile:
#        json.dump(global_segment_list, outfile)

# Aca tengo que volver a cargar el archivo de arriba y dejarlo como lista para traer lo otro

In [35]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,20230301_CWB_padariacafebolos_piloto_NB_aguase...,padariacafebolos_piloto_NB_aguaseisotonicos,7,true,01-03-2023,31-03-2023,9,1,None,391141#391142#391140
1,20230301_CWB_padariacafebolos_piloto_NB_cervej...,padariacafebolos_piloto_NB_cervejas,6,true,01-03-2023,31-03-2023,9,1,None,370805#370806#370807#370892#370891#370893#4940...
2,20230301_CWB_padariacafebolos_piloto_NB_refrig...,padariacafebolos_piloto_NB_refrigeranteseenerg...,7,true,01-03-2023,31-03-2023,9,1,None,453386#453385#370742#370740#370741#453373#4533...
3,20230301_CWB_padariacafebolos_piloto_NB_sucose...,padariacafebolos_piloto_NB_sucosechas,6,true,01-03-2023,31-03-2023,9,1,None,370778#370779#370775#370774#370762#370763
4,20230301_CWB_padariacafebolos_piloto_NB_destil...,padariacafebolos_piloto_NB_destilados,3,true,01-03-2023,31-03-2023,9,1,None,589485#589483


In [36]:
len_seg =int(len(global_segment_list)/10)
len_seg_list = [global_segment_list[:len_seg],global_segment_list[len_seg:len_seg*2],global_segment_list[len_seg*2:len_seg*3],global_segment_list[len_seg*3:len_seg*4],global_segment_list[len_seg*4:len_seg*5],global_segment_list[len_seg*5:len_seg*6],global_segment_list[len_seg*6:len_seg*7],global_segment_list[len_seg*7:len_seg*8],global_segment_list[len_seg*8:len_seg*9],global_segment_list[len_seg*9:]]

In [37]:
import requests
import json

def create_segments(segment_list):
    
    url = f"https://{city_code}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

    payload = json.dumps(segment_list)


    headers = {
    'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
    'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.status_code)
    sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]
    time.sleep(70)
    return sgt

In [38]:
sgt=[]

for i in len_seg_list:
    sgt.append(create_segments(i))
    
sgt_end =[item for lista in sgt for item in lista]

200
200
200
200
200
200
200
200
200
200


In [39]:
for i in global_offer_csv.offer_name:
    for j,z in sgt_end:
        if z==i:
            global_offer_csv.loc[global_offer_csv.offer_name == i, ('customer_segment_id')] = j

In [40]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,20230301_CWB_padariacafebolos_piloto_NB_aguase...,padariacafebolos_piloto_NB_aguaseisotonicos,7,true,01-03-2023,31-03-2023,9,1,125690,391141#391142#391140
1,20230301_CWB_padariacafebolos_piloto_NB_cervej...,padariacafebolos_piloto_NB_cervejas,6,true,01-03-2023,31-03-2023,9,1,125691,370805#370806#370807#370892#370891#370893#4940...
2,20230301_CWB_padariacafebolos_piloto_NB_refrig...,padariacafebolos_piloto_NB_refrigeranteseenerg...,7,true,01-03-2023,31-03-2023,9,1,125692,453386#453385#370742#370740#370741#453373#4533...
3,20230301_CWB_padariacafebolos_piloto_NB_sucose...,padariacafebolos_piloto_NB_sucosechas,6,true,01-03-2023,31-03-2023,9,1,125695,370778#370779#370775#370774#370762#370763
4,20230301_CWB_padariacafebolos_piloto_NB_destil...,padariacafebolos_piloto_NB_destilados,3,true,01-03-2023,31-03-2023,9,1,125693,589485#589483


In [41]:
global_offer_csv = global_offer_csv.reset_index(drop=True)
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('discount','max_uses_per_order')]=1
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('start_date','end_date')]='01-01-2024'
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('end_date')]='02-01-2024'
global_offer_csv.loc[global_offer_csv['offer_name'].str.contains("control", case=False), ('sku_id')]= global_offer_csv.loc[~global_offer_csv['offer_name'].str.contains("piloto", case=False), ('sku_id')].tolist()

In [42]:
len_offer =int(len(global_offer_csv)/4)
len_offer = [global_offer_csv[:len_seg],global_offer_csv[len_seg:len_seg*2],global_offer_csv[len_seg*2:len_seg*3],global_offer_csv[len_seg*3:]]

In [43]:
global_offer_csv.head()

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,20230301_CWB_padariacafebolos_piloto_NB_aguase...,padariacafebolos_piloto_NB_aguaseisotonicos,7,true,01-03-2023,31-03-2023,9,1,125690,391141#391142#391140
1,20230301_CWB_padariacafebolos_piloto_NB_cervej...,padariacafebolos_piloto_NB_cervejas,6,true,01-03-2023,31-03-2023,9,1,125691,370805#370806#370807#370892#370891#370893#4940...
2,20230301_CWB_padariacafebolos_piloto_NB_refrig...,padariacafebolos_piloto_NB_refrigeranteseenerg...,7,true,01-03-2023,31-03-2023,9,1,125692,453386#453385#370742#370740#370741#453373#4533...
3,20230301_CWB_padariacafebolos_piloto_NB_sucose...,padariacafebolos_piloto_NB_sucosechas,6,true,01-03-2023,31-03-2023,9,1,125695,370778#370779#370775#370774#370762#370763
4,20230301_CWB_padariacafebolos_piloto_NB_destil...,padariacafebolos_piloto_NB_destilados,3,true,01-03-2023,31-03-2023,9,1,125693,589485#589483


In [44]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'SD/export_data/MS/csv_{city_code}_All_MS.csv', index = False)

for i,offer in enumerate(len_offer):
    offer.to_csv(f'SD/export_data/MS/csv_{city_code}_All_MS_{i}.csv', index = False)

with open(f'SD/export_data/MS/nb_{city_code}_All_MS.txt', 'w') as outfile:
        json.dump(global_segment_list, outfile)    
with open(f'SD/export_data/MS/nb_{city_code}_All_MS.json', 'w') as outfile:
        json.dump(global_segment_list, outfile)
print(f'Done {city_code}')

Done CWB


# Esto es el experimento de R&D

In [45]:
# import ast

In [46]:
# with open('SD/export_data/MS/nb_BOG_All_MS.txt', 'r') as file:
#     data = file.read()
# data = ast.literal_eval(data)

In [47]:
# global_segment_list = data

In [48]:
# exp_ryd_1 = pd.read_csv(f"csv_{city_code}_CLIENTS_FINAL_ENERO_R&D-other_b.csv")#['costumer_id'].unique().tolist()
# exp_ryd_2 = pd.read_csv(f"csv_{city_code}_CLIENTS_FINAL_ENERO_R&D-other_a.csv")#['costumer_id'].unique().tolist()
# exp_ryd_3 = pd.read_csv(f"csv_{city_code}_CLIENTS_FINAL_ENERO_R&D_fruver.csv")#['costumer_id'].unique().tolist()

# exp_ryd_4 = pd.concat([exp_ryd_1, exp_ryd_2], ignore_index=True)

# exp_ryd = pd.concat([exp_ryd_3, exp_ryd_4], ignore_index=True)

In [49]:
# exp_ryd['previous_group'] = exp_ryd.previous_group.replace("Treatment","piloto")
# exp_ryd['previous_group'] = exp_ryd.previous_group.replace("Control","control")

# exp_ryd['final_group'] = exp_ryd.final_group.replace("Treatment","piloto")
# exp_ryd['final_group'] = exp_ryd.final_group.replace("Control","control")
# exp_ryd['offer_old'] = "20230101_"+city_code+"_"+exp_ryd.ms+"_"+exp_ryd.previous_group+"_NB_"+exp_ryd.subcategory+"_W52"
# exp_ryd['offer_new'] = "20230101_"+city_code+"_"+exp_ryd.ms+"_"+exp_ryd.final_group+"_NB_"+exp_ryd.subcategory+"_W52"

# exp_ryd['customer_source_id'] = exp_ryd.customer_source_id.astype(int)

# delete_old = exp_ryd[exp_ryd.offer_old != exp_ryd.offer_new].groupby('offer_old')['customer_source_id'].apply(list).reset_index(name='new')
# add_new = exp_ryd[exp_ryd.offer_old != exp_ryd.offer_new].groupby('offer_new')['customer_source_id'].apply(list).reset_index(name='new')

In [50]:
# for lis in delete_old.values:
#     for dic in global_segment_list:
#         if dic['name'] == lis[0]:
#             for adds in lis[1]:
#                 #print(dic['name'])
#                 try:
#                     dic['customersIds'].remove(adds)
#                 except:
#                     pass
#                 #print(dic['customersIds'])
#                 break
#             break
#     break

In [51]:
# for lis in add_new.values:
#     for dic in global_segment_list:
#         if dic['name'] == lis[0]:
#             for adds in lis[1]:
#                 dic['customersIds'].append(adds)
#                 #print(dic['customersIds'])
#                 break
#             break
#     break